# TorQ: Create Everything
This notebook will use the AWS boto3 APIs to create the needed resources for a TorQ based application.

## AWS Resources Created
- Database   
- Changeset to add data to database   
- Scaling Group that will contain all clusters   
- Shared Volume   
- Dataview of database on the shared volume   
- Clusters


In [1]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *

from env import *
from config import *

# ----------------------------------------------------------------

TORQ_CODEBASE="TorQ"
TORQ_FINSPACE_CODEBASE="TorQ-Amazon-FinSpace-Starter-Pack"

# Source data directory
SOURCE_DATA_DIR=f"{TORQ_FINSPACE_CODEBASE}/hdb"

# Code directory
CODEBASE="torq_app"

# unique code zipfile name
today=datetime.datetime.now().strftime("%Y%m%d_%H%M")    
CODE_ZIP=f"{CODEBASE}_{today}.zip"

# S3 Destinations
S3_CODE_PATH="code"
S3_DATA_PATH="data"

NODE_TYPE="kx.sg.4xlarge"

DATABASE_CONFIG=[{ 
    'databaseName': DB_NAME,
    'dataviewName': DBVIEW_NAME
    }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODE_ZIP}' }

NAS1_CONFIG= {
        'type': 'SSD_250',
        'size': 1200
}

#update_type="NO_RESTART"
#update_type="ROLLING"
update_type="FORCE"

In [2]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

# Stage Fresh Code to S3
Code to be used in this application must be staged to an S3 bucket the service can read from, that code will then be deployed to the clusters as part of their creation workflow.

In [3]:
# zip the code
os.system(f"zip -q -r {CODE_ZIP} TorQ/ TorQ-Amazon-FinSpace-Starter-Pack/ -x '*.ipynb_checkpoints*' -x 'TorQ-Amazon-FinSpace-Starter-Pack/hdb/*' -x '*.git*' -x 'TorQ/tests/*' -x 'TorQ-Amazon-FinSpace-Starter-Pack/terraform-deployment/*'")

cp = ""

# copy code to S3
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}
"""

cp += f"""
aws s3 cp --exclude .DS_Store {CODE_ZIP} s3://{S3_BUCKET}/code/{CODE_ZIP}
aws s3 ls s3://{S3_BUCKET}/code/{CODE_ZIP}
"""
    
# execute the S3 copy
os.system(cp)

upload: ./torq_app_20241203_2103.zip to s3://kdb-demo-829845998889-kms/code/torq_app_20241203_2103.zip
2024-12-03 21:03:48    8925181 torq_app_20241203_2103.zip


0

In [4]:
# update clusters in same order as they where created
# this will account for dependencies

for c in clusters:
    # wait for a cluster?
    if c['type'] == "WAIT":
        wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=c['name'], show_wait=True)
        continue

    # cluster must be running to update it
    wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=c['name'], show_wait=True)

    cluster_name = c['name']
    cluster_type = c['type']
    cluster_init = c['init']
    cluster_args = c['args']

    print(f"Updating: {cluster_name}")

    update_args = {}

    update_args['environmentId']=ENV_ID
    update_args['clusterName']=cluster_name 
    update_args['code']= CODE_CONFIG
    update_args['deploymentConfiguration']={
        'deploymentStrategy': update_type
    }

    if update_type != "NO_RESTART":
        update_args['initializationScript']=cluster_init
        update_args['commandLineArguments']=cluster_args

    display( update_args )

    resp = client.update_kx_cluster_code_configuration( **update_args )

    display(resp)

Cluster: discovery1 status is now RUNNING, total wait 0:00:00
Updating: discovery1


{'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'clusterName': 'discovery1',
 'code': {'s3Bucket': 'kdb-demo-829845998889-kms',
  's3Key': 'code/torq_app_20241203_2103.zip'},
 'deploymentConfiguration': {'deploymentStrategy': 'FORCE'},
 'initializationScript': 'TorQ-Amazon-FinSpace-Starter-Pack/finspace_torq.q',
 'commandLineArguments': [{'key': 'proctype', 'value': 'discovery'},
  {'key': 'procname', 'value': 'discovery1'},
  {'key': 'jsonlogs', 'value': 'true'},
  {'key': 'noredirect', 'value': 'true'},
  {'key': 's', 'value': '2'}]}

{'ResponseMetadata': {'RequestId': 'fb78a703-3e7b-40d1-ab30-841760e2fc6d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 21:03:50 GMT',
   'x-amzn-requestid': 'fb78a703-3e7b-40d1-ab30-841760e2fc6d',
   'x-amz-apigw-id': 'CO7IVHn4IAMEJ_w=',
   'x-amzn-trace-id': 'Root=1-674f7234-6661b7d404b6f1bd7f3484b0',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 e37edb91bbf6b4d78a917647aaf7d0d0.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'J2MA2ikfXkWCL2GDcJrMN0cHK-ei4dyAOaSQ0ensypzKVif6KoISTw=='},
  'RetryAttempts': 0}}

Cluster: rdb1 status is now RUNNING, total wait 0:00:00
Updating: rdb1


{'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'clusterName': 'rdb1',
 'code': {'s3Bucket': 'kdb-demo-829845998889-kms',
  's3Key': 'code/torq_app_20241203_2103.zip'},
 'deploymentConfiguration': {'deploymentStrategy': 'FORCE'},
 'initializationScript': 'TorQ-Amazon-FinSpace-Starter-Pack/finspace_torq.q',
 'commandLineArguments': [{'key': 'proctype', 'value': 'rdb'},
  {'key': 'procname', 'value': 'rdb1'},
  {'key': 'jsonlogs', 'value': 'true'},
  {'key': 'noredirect', 'value': 'true'},
  {'key': 's', 'value': '2'}]}

...

{'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'clusterName': 'feed1',
 'code': {'s3Bucket': 'kdb-demo-829845998889-kms',
  's3Key': 'code/torq_app_20241203_2103.zip'},
 'deploymentConfiguration': {'deploymentStrategy': 'FORCE'},
 'initializationScript': 'TorQ-Amazon-FinSpace-Starter-Pack/finspace_torq.q',
 'commandLineArguments': [{'key': 'proctype', 'value': 'tradeFeed'},
  {'key': 'procname', 'value': 'tradeFeed1'},
  {'key': 'jsonlogs', 'value': 'true'},
  {'key': 'noredirect', 'value': 'true'},
  {'key': 's', 'value': '2'}]}

{'ResponseMetadata': {'RequestId': '71e012e8-1881-495b-8b1b-076f1e73c082',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 21:14:58 GMT',
   'x-amzn-requestid': '71e012e8-1881-495b-8b1b-076f1e73c082',
   'x-amz-apigw-id': 'CO8w6GT8IAMEikg=',
   'x-amzn-trace-id': 'Root=1-674f74d2-13da01ed3c554caa7bf6d728',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 e37edb91bbf6b4d78a917647aaf7d0d0.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'De1408uwUtrp_tSlKt1-gQu2ZlkFckEp7ZVGRKxotuLZ3Oaq5YsRnA=='},
  'RetryAttempts': 0}}

Cluster: monitor1 status is now RUNNING, total wait 0:00:00
Updating: monitor1


{'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'clusterName': 'monitor1',
 'code': {'s3Bucket': 'kdb-demo-829845998889-kms',
  's3Key': 'code/torq_app_20241203_2103.zip'},
 'deploymentConfiguration': {'deploymentStrategy': 'FORCE'},
 'initializationScript': 'TorQ-Amazon-FinSpace-Starter-Pack/finspace_torq.q',
 'commandLineArguments': [{'key': 'proctype', 'value': 'monitor'},
  {'key': 'procname', 'value': 'monitor1'},
  {'key': 'jsonlogs', 'value': 'true'},
  {'key': 'noredirect', 'value': 'true'},
  {'key': 's', 'value': '1'}]}

{'ResponseMetadata': {'RequestId': '527ecf5d-e028-42a5-a606-954f7e6d102b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'date': 'Tue, 03 Dec 2024 21:14:59 GMT',
   'x-amzn-requestid': '527ecf5d-e028-42a5-a606-954f7e6d102b',
   'x-amz-apigw-id': 'CO8xAGsyIAMEXjQ=',
   'x-amzn-trace-id': 'Root=1-674f74d2-64d640840a5b6ee05c322c88',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 e37edb91bbf6b4d78a917647aaf7d0d0.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'uvbeCLO6b8FJkK7vrcCrBlTpZAfQ5IdjTEAe_3Vx_90EX0j4zo8-IA=='},
  'RetryAttempts': 0}}

## Wait for all clusters to finish updating

In [5]:
# Wait for all clusters to start
for c in clusters:
    cluster_name = c['name']
    wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=cluster_name, show_wait=True)

print("** ALL DONE **")

Cluster: discovery1 status is now RUNNING, total wait 0:00:00
Cluster: rdb1 status is now RUNNING, total wait 0:00:00
Cluster: hdb1 status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Cluster: hdb1 status is now RUNNING, total wait 0:00:30
Cluster: discovery1 status is now RUNNING, total wait 0:00:00
Cluster: rdb1 status is now RUNNING, total wait 0:00:00
Cluster: gateway1 status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:00:30, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:01:00, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:01:30, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:02:00, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:02:30, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:03:00, waiting 30 sec ...
Cluster: gateway1 status is UPDATING, total wait 0:03:30, waiting 30 sec ...
Cluster: gateway1 s

# List Clusters

In [6]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = []

for c in clusters:
    all_clusters.append(c['name'])

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
3,discovery1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'discovery1'}, {'key': 'proctype', 'value': 'discovery'}]",Created with create_all notebook,2024-12-03 21:14:44.294000+00:00,2024-12-03 20:01:35.746000+00:00,finspace-database,None
4,feed1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'tradeFeed1'}, {'key': 'proctype', 'value': 'tradeFeed'}]",Created with create_all notebook,2024-12-03 21:27:04.539000+00:00,2024-12-03 20:18:27.837000+00:00,finspace-database,None
5,gateway1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'gateway1'}, {'key': 'proctype', 'value': 'gateway'}]",Created with create_all notebook,2024-12-03 21:27:44.434000+00:00,2024-12-03 20:18:24.649000+00:00,finspace-database,None
6,hdb1,RUNNING,GP,None,"[{'key': 's', 'value': '4'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'hdb1'}, {'key': 'proctype', 'value': 'hdb'}]",Created with create_all notebook,2024-12-03 21:15:12.760000+00:00,2024-12-03 20:01:41.735000+00:00,finspace-database,None
7,monitor1,RUNNING,GP,None,"[{'key': 's', 'value': '1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'monitor1'}, {'key': 'proctype', 'value': 'monitor'}]",Created with create_all notebook,2024-12-03 21:25:26.933000+00:00,2024-12-03 20:18:30.995000+00:00,finspace-database,None
8,rdb1,RUNNING,GP,None,"[{'key': 's', 'value': '2'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 'procname', 'value': 'rdb1'}, {'key': 'proctype', 'value': 'rdb'}]",Created with create_all notebook,2024-12-03 21:14:43.708000+00:00,2024-12-03 20:01:39.007000+00:00,finspace-database,None


# Logs
You can use AWS contols and log trail to monitor logs, below are terminal based ways to tail the log as well for each cluster.

In [7]:
def get_log_group(environmentId: str, clusterName: str):
    return f"/aws/vendedlogs/finspace/{environmentId}/{clusterName}"

#log_format="detailed"
#log_format="short"
log_format="json"

print("Copy/Paste any line below in a terminal with AWS CLI to tail the logs of that cluster")
print()
for c in clusters:
    cluster_name = c['name']

    lg = get_log_group(environmentId=ENV_ID, clusterName=cluster_name)

    print( f"aws logs tail {lg} --follow --format {log_format}" )

Copy/Paste any line below in a terminal with AWS CLI to tail the logs of that cluster

aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/discovery1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/rdb1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/hdb1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/discovery1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/rdb1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/gateway1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/feed1 --follow --format json
aws logs tail /aws/vendedlogs/finspace/jlcenjvtkgzrdek2qqv7ic/monitor1 --follow --format json


# All Processes Running

In [8]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-12-03 21:28:07.699778
